# Import Dependencies

In [3]:

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
     ---------------------------------------- 4.9/4.9 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 GB 882.2 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [4]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

# initiate Model 

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

# Collect reviews online

In [7]:
r = requests.get('https://www.yelp.com/biz/burger-king-san-francisco-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]# removing the html tags and considering only text 

In [9]:
reviews[0]

'They have an extraordinary delivery service. With delicious burgers and fries. I like their fast delivery service and quality.'

# load reviews into DataFrame 

In [10]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [11]:
df['review'].iloc[0]

'They have an extraordinary delivery service. With delicious burgers and fries. I like their fast delivery service and quality.'

# Model predection

In [12]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [13]:
sentiment_score(df['review'].iloc[1])

1

In [14]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [15]:
df

,review,sentiment
0,They have an extraordinary delivery service. W...,5
1,What happened to innovation in burger chefery?...,1
2,Ate the Spiderman burger thought I was going t...,2
3,My on-line orders sat for forty five minutes b...,1
4,Place is absolutely filthy and the slowest bur...,1
5,Omg! Little mouse running arround the dining r...,1
6,"My mom, grandma , and uncle came in trying to ...",2
7,Horrible Customer service.... I have been wait...,1
8,Great still to this day order the impossible b...,5
9,burger king took my friends money and didnt gi...,1


In [16]:
df['review'].iloc[3]

"My on-line orders sat for forty five minutes before I had to go up to the counter and ask where they were. The counter intelligence didn't call my number or name and the burgers and fries were cold. The kitchen help didn't know or understand English or how to make a correct double whopper from a single whopper. A muni driver received the wrong burger and had to come back for the correct one. The unintelligent counter help fucked up my order, and I spent over $45 for three burgers, three fries, three drinks and one dessert. Horrible service and nasty tasting fries. The on-line custom service gave a $5 coupon, as it it were an incentive to return. Only positive? They serve impossible burgers."

# Import downloaded reviews from csv file 

In [17]:
df_loaded=pd.read_csv(r"C:\Users\mgalaval\Downloads\Reviews.csv\Reviews.csv")

# Model predection

In [25]:
sentiment_score(df_loaded['Text'].iloc[1])

1

In [ ]:
df_loaded['sentiment'] = df_loaded['Text'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df_loaded